# 1. Import Modules

In [0]:
import numpy as np
import torch
import os
import pickle

import torch.nn.functional as F

from torch import nn
from torch.utils.data import TensorDataset, DataLoader

import helper
# import problem_unittests as tests

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
SPECIAL_WORDS = {'PADDING': '<PAD>'}

# 2. Explore the Data

In [0]:
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data

In [0]:
def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    """
    Preprocess Text Data
    """
    text = load_data(dataset_path)
    
    # Ignore notice, since we don't use it for analysing the data
    text = text[81:]

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text + list(SPECIAL_WORDS.values()))
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('gdrive/My Drive/Lab/preprocess.p', 'wb'))

In [0]:
def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('gdrive/My Drive/Lab/preprocess.p', mode='rb'))

In [0]:
def save_model(filename, decoder):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)

In [0]:
data_dir = 'gdrive/My Drive/Lab/data/Seinfeld_Scripts.txt'
text = load_data(data_dir)

In [9]:
view_line_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

# 3. Implement Pre-processing Functions

### Lookup Table

In [0]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    vocab_to_int = dict()
    int_to_vocab = dict()
    
    sorted_word_set = sorted(set(text))
    
    for i, word in enumerate(sorted_word_set):
        vocab_to_int[word] = i
        int_to_vocab[i] = word
    
    return (vocab_to_int, int_to_vocab)

In [0]:
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation

In [0]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    token_dict = dict()
    
    punctuation_list = [
        '.', 
        ',', 
        '"', 
        ';', 
        '!', 
        '?', 
        '(', 
        ')',
        '-',
        '\n'
    ]
    
    token_list = [
        '||Period||', 
        '||Comma||', 
        '||Quotation_Mark||', 
        '||Semicolon||', 
        '||Exclamation_Mark||', 
        '||Question_Mark||', 
        '||Left_Parentheses||', 
        '||Right_Parentheses||', 
        '||Dash||', 
        '||Return||'
    ]
    
    for (punctuation, token) in zip(punctuation_list, token_list):
        token_dict[punctuation] = token
        
    return token_dict

In [0]:
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process & Save Data

In [0]:
preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# **************** #1 Check Point ****************

In [0]:
int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

### Check Access to GPU

In [0]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

### Batch Input Data

In [0]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    
    ###
    
    words_len = len(words)
    
    batch_size_total = batch_size * sequence_length
    
    n_batches = words_len // batch_size_total
    
    words = words[:n_batches * batch_size_total]
    
    x,y = [],[]
    
    for idx in range(0,len(words) - sequence_length):
        x.append(words[idx:idx + sequence_length])
        y.append(words[idx+sequence_length])
        
    feature_arr = np.asarray(x)
    target_arr = np.asarray(y)
        
    ###
    
    feature_tensors = torch.from_numpy(feature_arr)
    target_tensors = torch.from_numpy(target_arr)
    
    ###
    
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(
        data, 
        shuffle=True,
        batch_size=batch_size
    )
        
        
    return data_loader

In [16]:
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=2)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([2, 5])
tensor([[ 9, 10, 11, 12, 13],
        [44, 45, 46, 47, 48]])

torch.Size([2])
tensor([14, 49])


# 4. Build the Neural Network

In [0]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.input_dim = vocab_size
        self.output_dim = output_size
        self.n_hidden = hidden_dim
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers
        
        # define model layers
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_dim, 
            batch_first=True, 
            num_layers=n_layers,
            dropout=dropout)
        self.linear = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(dropout)
        
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        batch_size = nn_input.size(0)
        
        embeds = self.embeddings(nn_input)
        
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = self.dropout(lstm_out)
        
        lstm_out = lstm_out.contiguous().view(-1, self.n_hidden)
        
        output = self.linear(lstm_out)
        
        output = output.view(batch_size, -1, self.output_dim)
        
        out = output[:, -1]
        
        return out, hidden

In [0]:
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

In [0]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    hidden = tuple([each.data for each in hidden])
    
    # zero accumulated gradients
    rnn.zero_grad()
    
    # get the output from the model
    output, hidden = rnn(inp, hidden)
    
    # calculate the loss and perform backprop
    loss = criterion(output, target)
    loss.backward()
     
    optimizer.step()
    
    return loss.item(), hidden

In [0]:
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


# 5. Neural Network Training

### Train Loop

In [0]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

In [0]:
# Data params
# Sequence Length
sequence_length = 16  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

# Training parameters
# Number of Epochs
num_epochs = 50
# Learning Rate
learning_rate = 0.0005

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 250
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train

In [21]:
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
save_model('gdrive/My Drive/Lab/save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 50 epoch(s)...
Epoch:    1/50    Loss: 5.711249412536621

Epoch:    1/50    Loss: 5.082968925476075

Epoch:    1/50    Loss: 4.919073328971863

Epoch:    1/50    Loss: 4.797498863697052

Epoch:    1/50    Loss: 4.697561295509338

Epoch:    1/50    Loss: 4.624488952636718

Epoch:    1/50    Loss: 4.586837390422821

Epoch:    1/50    Loss: 4.527997499465942

Epoch:    1/50    Loss: 4.470246723175049

Epoch:    1/50    Loss: 4.4731695203781126

Epoch:    1/50    Loss: 4.466984477043152

Epoch:    1/50    Loss: 4.4267758660316465

Epoch:    1/50    Loss: 4.4036310458183285

Epoch:    2/50    Loss: 4.347306964544111

Epoch:    2/50    Loss: 4.294382278442383

Epoch:    2/50    Loss: 4.285514365196228

Epoch:    2/50    Loss: 4.256610577106476

Epoch:    2/50    Loss: 4.245897693157196

Epoch:    2/50    Loss: 4.264202850341797

Epoch:    2/50    Loss: 4.247508197307587

Epoch:    2/50    Loss: 4.23816818523407

Epoch:    2/50    Loss: 4.244073395252228

Epoch:    2/50    Loss: 

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
